[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pinecone-io/examples/blob/master/learn/generation/langchain/rag-chatbot.ipynb) [![Open nbviewer](https://raw.githubusercontent.com/pinecone-io/examples/master/assets/nbviewer-shield.svg)](https://nbviewer.org/github/pinecone-io/examples/blob/master/learn/generation/langchain/rag-chatbot.ipynb)

In [1]:
# %pip install dspy-ai

In [2]:
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv()) # read local .env file

In [3]:
!mkdir -p 'data'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt' -O 'data/paul_graham_essay.txt'

--2024-03-29 16:48:54--  https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 75042 (73K) [text/plain]
Saving to: ‘data/paul_graham_essay.txt’

data/paul_graham_es 100%[===================>]  73.28K  --.-KB/s    in 0.01s   

2024-03-29 16:48:54 (5.60 MB/s) - ‘data/paul_graham_essay.txt’ saved [75042/75042]



In [4]:
# %pip install -U weaviate-client

In [5]:
import weaviate
from weaviate.embedded import EmbeddedOptions
import os

# Connect to Weaviate client in embedded mode
client = weaviate.Client(embedded_options=EmbeddedOptions(),
                             additional_headers={
                                "X-OpenAI-Api-Key": os.environ.get('OPENAI_API_KEY'),
                             }
                         )


embedded weaviate is already listening on port 8079


/Users/keunbae/code/langchain-rag-tutorial/.venv/lib/python3.11/site-packages/weaviate/warnings.py:158: DeprecationWarning: Dep016: You are using the Weaviate v3 client, which is deprecated.
            Consider upgrading to the new and improved v4 client instead!
            See here for usage: https://weaviate.io/developers/weaviate/client-libraries/python
            
  warnings.warn(


In [6]:

# Create Weaviate schema
# DSPy assumes the collection has a text key 'content'
schema = {
   "classes": [
       {
           "class": "MyExampleIndex",
           "vectorizer": "text2vec-openai",
            "moduleConfig": {"text2vec-openai": {}},
           "properties": [{"name": "content", "dataType": ["text"]}]
       }      
   ]
}
    
# client.schema.create(schema)


In [7]:
import re

# Split document into single sentences
chunks = []
with open("./data/paul_graham_essay.txt", 'r', encoding='utf-8') as file:
    text = file.read()
    # split by sentence
    sentences = re.split(r'[\r\n]+', text)
    sentences = [sentence.strip() for sentence in sentences if sentence.strip()]
    chunks.extend(sentences)


In [8]:

# Populate vector database in batches
client.batch.configure(batch_size=100)  # Configure batch

with client.batch as batch:  # Initialize a batch process
    for i, d in enumerate(chunks):  # Batch import data
        properties = {
            "content": d,
        }
        batch.add_data_object(
            data_object=properties,
            class_name="MyExampleIndex"
        )

---